In [21]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from skimage.feature import hog, local_binary_pattern
from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgb2gray
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import os
import cv2

In [22]:
#Initialize Global Variables
data = []
# data_dir = '../dataset/train_data'
data_dir = '../../../notebooks/sushant07/raw_image/test'

In [23]:
# Tile image to hog features extraction
def extract_hog_features(file):
    file_path = os.path.join(data_dir, file)
    image = imread(file_path)
    gray_image = rgb2gray(image)
    resized_image = resize(gray_image, (800, 600), anti_aliasing=True)
    
    # Get image dimensions
    height, width = resized_image.shape
    
    # Calculate tile size
    tile_height = height // 8
    tile_width = width // 8
    
    # Create figure for 8x8 subplot
    # fig, axes = plt.subplots(8, 8, figsize=(12, 12))
    # fig.suptitle(f'Tiles for {file}', fontsize=16)
    # Dictionary to store features for this image
    hog_features = {}
    # Split image into 8x8 tiles
    for i in range(8):
        for j in range(8):
            # Calculate tile boundaries
            y_start = i * tile_height
            y_end = (i + 1) * tile_height
            x_start = j * tile_width
            x_end = (j + 1) * tile_width
            
            # Extract tile
            tile = resized_image[y_start:y_end, x_start:x_end]
            
            # Display tile in subplot
            # axes[i, j].imshow(tile, cmap='gray')
            # axes[i, j].axis('off')
            # axes[i, j].set_title(f'{i},{j}', fontsize=6)
            
            # Extract HOG features from tile
            features = hog(tile, orientations=9,
                        pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2),
                        block_norm='L2-Hys')
            # Store in dictionary with key: (image_name, i, j)
            key = (file, i, j)
            hog_features[key] = features
            
            # Also append to global data list
            data.append({
                'type': 'hog',
                'image': file,
                'tile_i': i,
                'tile_j': j,
                'features': features
            })
    # plt.tight_layout()
    # plt.show()
    return hog_features

In [24]:
def extract_lbp_features(file):
    file_path = os.path.join(data_dir, file)
    image = imread(file_path)
    gray_image = rgb2gray(image)
    resized_image = resize(gray_image, (800, 600), anti_aliasing=True)
    # Get image dimensions
    height, width = resized_image.shape
    # Calculate tile size
    tile_height = height // 8
    tile_width = width // 8
    # Dictionary to store features for this image
    lbp_features = {}
    # Split image into 8x8 tiles
    for i in range(8):
        for j in range(8):
            # Calculate tile boundaries
            y_start = i * tile_height
            y_end = (i + 1) * tile_height
            x_start = j * tile_width
            x_end = (j + 1) * tile_width
            
            # Extract tile
            tile = resized_image[y_start:y_end, x_start:x_end]
            tile_uint8 = (tile * 255).astype(np.uint8)
            # Extract LBP features from tile
            P = 8
            R = 1
            n_bins = P + 2
            lbp = local_binary_pattern(tile_uint8, P=P, R=R, method="uniform")
            # lbp = local_binary_pattern(tile, n_points, radius, method='uniform')
            (hist, _) = np.histogram(lbp.ravel(),
                           bins=n_bins,
                           range=(0, n_bins),
                           density=True)
            hist = hist.astype("float")
            hist /= (hist.sum() + 1e-7)
            
            # Store in dictionary with key: (image_name, i, j)
            key = (file, i, j)
            lbp_features[key] = hist
            
            # Also append to global data list
            data.append({
                'type': 'lbp',
                'image': file,
                'tile_i': i,
                'tile_j': j,
                'features': hist
            })
    # plt.tight_layout()
    # plt.show()
    return lbp_features

In [25]:
def plot_color_histogram(file, tile_i, tile_j, tile):
    """Plot color histograms in 2x2 subplot"""
    fig, axes = plt.subplots(2, 2, figsize=(10, 8))
    fig.suptitle(f'{file} - Tile ({tile_i},{tile_j})', fontsize=14)
    
    # Compute histograms for each channel
    hist_r, bins_r = np.histogram(tile[:, :, 0], bins=32, range=(0, 1))
    hist_g, bins_g = np.histogram(tile[:, :, 1], bins=32, range=(0, 1))
    hist_b, bins_b = np.histogram(tile[:, :, 2], bins=32, range=(0, 1))
    
    # Top-left: All channels combined
    axes[0, 0].plot(bins_r[:-1], hist_r, color='red', alpha=0.7, label='Red')
    axes[0, 0].plot(bins_g[:-1], hist_g, color='green', alpha=0.7, label='Green')
    axes[0, 0].plot(bins_b[:-1], hist_b, color='blue', alpha=0.7, label='Blue')
    axes[0, 0].set_title('All Channels')
    axes[0, 0].set_xlabel('Pixel Intensity')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Top-right: Red channel
    axes[0, 1].fill_between(bins_r[:-1], hist_r, color='red', alpha=0.7)
    axes[0, 1].set_title('Red Channel')
    axes[0, 1].set_xlabel('Pixel Intensity')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Bottom-left: Green channel
    axes[1, 0].fill_between(bins_g[:-1], hist_g, color='green', alpha=0.7)
    axes[1, 0].set_title('Green Channel')
    axes[1, 0].set_xlabel('Pixel Intensity')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Bottom-right: Blue channel
    axes[1, 1].fill_between(bins_b[:-1], hist_b, color='blue', alpha=0.7)
    axes[1, 1].set_title('Blue Channel')
    axes[1, 1].set_xlabel('Pixel Intensity')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].grid(True, alpha=0.3)
    
    # plt.tight_layout()
    # plt.show()

In [26]:
def extract_color_histogram_features(file):
    file_path = os.path.join(data_dir, file)
    image = imread(file_path)
    resized_image = resize(image, (800, 600), anti_aliasing=True)
    # Get image dimensions
    height, width, _ = resized_image.shape
    # Calculate tile size
    tile_height = height // 8
    tile_width = width // 8
    # Dictionary to store features for this image
    color_hist_features = {}
    # Split image into 8x8 tiles
    for i in range(8):
        for j in range(8):
            # Calculate tile boundaries
            y_start = i * tile_height
            y_end = (i + 1) * tile_height
            x_start = j * tile_width
            x_end = (j + 1) * tile_width
            
            # Extract tile
            tile = resized_image[y_start:y_end, x_start:x_end]
            
            # Compute color histogram for each channel and concatenate
            hist_r, _ = np.histogram(tile[:, :, 0], bins=32, range=(0, 1))
            hist_g, _ = np.histogram(tile[:, :, 1], bins=32, range=(0, 1))
            hist_b, _ = np.histogram(tile[:, :, 2], bins=32, range=(0, 1))
            hist = np.concatenate([hist_r, hist_g, hist_b]).astype("float")
            hist /= (hist.sum() + 1e-7)
            
            # Store in dictionary with key: (image_name, i, j)
            key = (file, i, j)
            color_hist_features[key] = hist
            # Plot histogram for first tile only (optional)
            if i == 0 and j == 0:
                plot_color_histogram(file, i, j, tile)
            # Also append to global data list
            data.append({
                'type': 'color_histogram',
                'image': file,
                'tile_i': i,
                'tile_j': j,
                'features': hist
            })
    return color_hist_features

In [27]:
def extract_orb_features(file, n_features=500):
    """
    Extract ORB features from image tiles.
    ORB is rotation-invariant and efficient for object detection.
    
    Parameters:
    - file: Image filename
    - n_features: Number of ORB keypoints to detect (default 500)
    """
    file_path = os.path.join(data_dir, file)
    image = imread(file_path)
    
    # Convert to grayscale for ORB detection
    gray_image = rgb2gray(image)
    resized_image = resize(gray_image, (800, 600), anti_aliasing=True)
    
    # Convert to 8-bit format for OpenCV
    resized_image_uint8 = (resized_image * 255).astype(np.uint8)
    
    # Get image dimensions
    height, width = resized_image_uint8.shape
    
    # Calculate tile size
    tile_height = height // 8
    tile_width = width // 8
    
    # Initialize ORB detector
    orb = cv2.ORB_create(nfeatures=n_features)
    
    # Dictionary to store ORB features
    orb_features = {}
    
    print(f"Extracting ORB features from {file}...")
    
    # Split image into 8x8 tiles
    for i in range(8):
        for j in range(8):
            # Calculate tile boundaries
            y_start = i * tile_height
            y_end = (i + 1) * tile_height
            x_start = j * tile_width
            x_end = (j + 1) * tile_width
            
            # Extract tile
            tile = resized_image_uint8[y_start:y_end, x_start:x_end]
            
            # Detect keypoints and compute descriptors
            keypoints, descriptors = orb.detectAndCompute(tile, None)
            
            # Handle case where no keypoints are detected
            if descriptors is None:
                # Create zero vector of expected size
                # ORB descriptor is 32 bytes (256 bits)
                descriptors = np.zeros((1, 32), dtype=np.uint8)
            
            # Flatten descriptor to 1D array for storage
            if len(keypoints) > 0:
                # Create feature vector: [num_keypoints, keypoint_response_mean, descriptor_mean]
                num_keypoints = len(keypoints)
                keypoint_responses = np.array([kp.response for kp in keypoints])
                response_mean = np.mean(keypoint_responses)
                response_std = np.std(keypoint_responses) if len(keypoint_responses) > 1 else 0
                
                # Compute descriptor statistics
                descriptor_mean = np.mean(descriptors, axis=0)
                descriptor_std = np.std(descriptors, axis=0)
                
                # Combine all features into single vector
                orb_feature_vector = np.concatenate([
                    [num_keypoints, response_mean, response_std],
                    descriptor_mean,
                    descriptor_std
                ])
            else:
                # If no keypoints, create zero vector
                orb_feature_vector = np.zeros(67)  # 3 + 32 + 32
            
            # Store in dictionary
            key = (file, i, j)
            orb_features[key] = orb_feature_vector
            
            # Append to global data list
            data.append({
                'type': 'orb',
                'image': file,
                'tile_i': i,
                'tile_j': j,
                'num_keypoints': len(keypoints),
                'features': orb_feature_vector
            })
    
    print(f"Completed ORB extraction for {file}")
    return orb_features


def visualize_orb_features(file, tile_i, tile_j, n_features=500):
    """
    Visualize ORB keypoints on a specific tile
    """
    file_path = os.path.join(data_dir, file)
    image = imread(file_path)
    gray_image = rgb2gray(image)
    resized_image = resize(gray_image, (800, 600), anti_aliasing=True)
    resized_image_uint8 = (resized_image * 255).astype(np.uint8)
    
    height, width = resized_image_uint8.shape
    tile_height = height // 8
    tile_width = width // 8
    
    # Extract specific tile
    y_start = tile_i * tile_height
    y_end = (tile_i + 1) * tile_height
    x_start = tile_j * tile_width
    x_end = (tile_j + 1) * tile_width
    
    tile = resized_image_uint8[y_start:y_end, x_start:x_end]
    
    # Detect ORB features
    orb = cv2.ORB_create(nfeatures=n_features)
    keypoints, descriptors = orb.detectAndCompute(tile, None)
    
    # Draw keypoints
    tile_with_keypoints = cv2.drawKeypoints(tile, keypoints, None, 
                                           color=(0, 255, 0),
                                           flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    # Display
    plt.figure(figsize=(10, 8))
    plt.imshow(tile_with_keypoints, cmap='gray')
    plt.title(f'{file} - Tile ({tile_i},{tile_j}) - ORB Keypoints: {len(keypoints)}')
    plt.axis('off')
    plt.show()

In [28]:
def write_features_to_file(filename='features_data_test_pexels.csv'):
    # Group data by (image, tile_i, tile_j)
    grouped_data = {}
    
    for item in data:
        key = (item['image'], item['tile_i'], item['tile_j'])
        if key not in grouped_data:
            grouped_data[key] = {
                'image': item['image'],
                'tile_i': item['tile_i'],
                'tile_j': item['tile_j'],
                'hog': None,
                'lbp': None,
                'orb': None
            }
        
        # Store features by type
        grouped_data[key][item['type']] = item['features']
    
    # Create records with combined features
    records = []
    for key, tile_data in grouped_data.items():
        record = {
            'image': tile_data['image'],
            'tile_i': tile_data['tile_i'],
            'tile_j': tile_data['tile_j']
        }
        
        # Add HOG features
        if tile_data['hog'] is not None:
            for idx, val in enumerate(tile_data['hog']):
                record[f'x_hog_{idx}'] = val
        
        # Add LBP features
        if tile_data['lbp'] is not None:
            for idx, val in enumerate(tile_data['lbp']):
                record[f'x_lbp_{idx}'] = val
        
        # Add Color Histogram features
        if tile_data['orb'] is not None:
            for idx, val in enumerate(tile_data['orb']):
                record[f'x_orb_{idx}'] = val
        
        records.append(record)
    
    # Create DataFrame
    df = pd.DataFrame(records)
    
    # Save to CSV
    df.to_csv(filename, index=False)
    
    print(f'Saved {len(df)} records to {filename}')
    print(f'Shape: {df.shape}')
    print(f'Columns: {df.columns[:10].tolist()}...')  # Show first 10 columns

In [29]:
#Run the extraction
for file in os.listdir(data_dir):
    hog_features = extract_hog_features(file)
    print(f'Extracted HOG features for {file}, total tiles: {len(hog_features)} and size of each tile feature vector: {len(next(iter(hog_features.values())))}')
    lbp_features = extract_lbp_features(file)
    print(f'Extracted LBP features for {file}, total tiles: {len(lbp_features)} and size of each tile feature vector: {len(next(iter(lbp_features.values())))}')
    # Extract ORB features (NEW)
    # orb_features = extract_orb_features(file, n_features=500)
    # print(f'Extracted ORB features for {file}: {len(orb_features)} tiles')

write_features_to_file()

Extracted HOG features for pexels.jpg, total tiles: 64 and size of each tile feature vector: 3168
Extracted LBP features for pexels.jpg, total tiles: 64 and size of each tile feature vector: 10
Saved 64 records to features_data_test_pexels.csv
Shape: (64, 3181)
Columns: ['image', 'tile_i', 'tile_j', 'x_hog_0', 'x_hog_1', 'x_hog_2', 'x_hog_3', 'x_hog_4', 'x_hog_5', 'x_hog_6']...


In [30]:
# # Read data from features_data_grp_1.csv
# import pandas as pd
# df_labelled = pd.read_csv('../outputs/features_data_labelled.csv')

# # print(df.head())
# print(df_labelled.shape)

# #drop columns image, tile_i, tile_j
# df_features = df_labelled.drop(columns=['image', 'tile_i', 'tile_j','y'])

# # find out correlations coefficients of all features with each other
# correlation_matrix = df_features.corr()

# #find out features which have correlation coefficient > 0.90 with each other
# high_correlation_pairs = []
# for i in range(len(correlation_matrix.columns)):
#     for j in range(i):
#         if abs(correlation_matrix.iloc[i, j]) > 0.90:
#             high_correlation_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], correlation_matrix.iloc[i, j]))
# print(f'Number of highly correlated feature pairs (>0.90): {len(high_correlation_pairs)}')
# #print first 10 highly correlated feature pairs
# for pair in high_correlation_pairs[:10]:
#     print(pair)

# # find unique features from the highly correlated pairs
# unique_features_to_drop = set()
# for feat1, feat2, corr in high_correlation_pairs:
#     unique_features_to_drop.add(feat2)  # arbitrarily drop the second feature in the pair
# print(f'Number of unique features to drop due to high correlation: {len(unique_features_to_drop)}')

# #drop these features from df_features
# df_reduced = df_labelled.drop(columns=list(unique_features_to_drop))
# print(f'Shape of features after dropping highly correlated ones: {df_reduced.shape}')

# #create csv of reduced features
# df_reduced.to_csv('../outputs/features_data_reduced.csv', index=False)